In [1]:
from datasets import load_dataset

dataset = load_dataset('Kili/plastic_in_river', num_proc=6)

print(dataset)

/Users/kevin/miniconda3/envs/cos429/lib/python3.8/site-packages/datasets/load.py:1486: FutureWarning: The repository for Kili/plastic_in_river contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Kili/plastic_in_river
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['image', 'litter'],
        num_rows: 3407
    })
    test: Dataset({
        features: ['image', 'litter'],
        num_rows: 427
    })
    validation: Dataset({
        features: ['image', 'litter'],
        num_rows: 425
    })
})
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1280x720 at 0x7FED38F83970>, 'litter': {'label': [2, 2], 'bbox': [[0.6937950849533081, 0.17073695361614227, 0.017922647297382355, 0.011738809756934643], [0.5574886202812195, 0.18079878389835358, 0.021695835515856743, 0.010061836801469326]]}}


In [ ]:
import os
from tqdm.notebook import tqdm  # Import tqdm from tqdm.notebook module

# only creating datasets for train and validation not test
os.makedirs('datasets/images/train', exist_ok=True)
os.makedirs('datasets/images/validation', exist_ok=True)
os.makedirs('datasets/labels/train', exist_ok=True)
os.makedirs('datasets/labels/validation', exist_ok=True)

def create_dataset(data, split):
  data = data[split]
  
  print(f'Running for {split} split...')
  
  for idx, sample in enumerate(data):
    image = sample['image']
    labels = sample['litter']['label']
    bboxes = sample['litter']['bbox']
    targets = []
    
    # creating the label txt files
    for label, bbox in zip(labels, bboxes):
      targets.append(f'{label} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}')
      
    with open(f'datasets/labels/{split}/{idx}.txt', 'w') as f:
      for target in targets:
        f.write(target+'\n')
        
    # saving image to png
    image.save(f'datasets/images/{split}/{idx}.png')

# create_dataset(dataset, 'train')
# create_dataset(dataset, 'validation')

Running for train split...
Running for validation split...


In [2]:
# !pip3 install --upgrade ultralytics
# !pip3 install opencv-python==4.8.0.74
# %conda install ipykernel --update-deps --force-reinstall

  Using cached opencv_python-4.9.0.80-cp37-abi3-macosx_10_16_x86_64.whl.metadata (20 kB)
Using cached opencv_python-4.9.0.80-cp37-abi3-macosx_10_16_x86_64.whl (55.7 MB)
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.5.5.64
    Uninstalling opencv-python-4.5.5.64:
      Successfully uninstalled opencv-python-4.5.5.64
  Using cached opencv_python-4.8.0.74-cp37-abi3-macosx_10_16_x86_64.whl.metadata (19 kB)
Using cached opencv_python-4.8.0.74-cp37-abi3-macosx_10_16_x86_64.whl (54.6 MB)
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.9.0.80
    Uninstalling opencv-python-4.9.0.80:
      Successfully uninstalled opencv-python-4.9.0.80


In [1]:
from ultralytics import YOLO # using YOLO!
import modal

app = modal.App()

model = YOLO('yolov8m.pt') # yolov8 architecture

@app.function(gpu="A100")
def run_obj_detection():
    model.train(
        data = 'plastic.yaml', # this plastic.yaml is the the config file for the object detection 
        epochs = 1, # relatively low for now just for testing
        imgsz = (1280, 720), # width, height
        batch = 4,
        optimizer = 'Adam',
        lr0 = 1e-3,
        device='cpu'
    )
    
run_obj_detection()

Ultralytics YOLOv8.2.3 🚀 Python-3.8.18 torch-2.2.2 CPU (Apple M2)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=plastic.yaml, epochs=1, time=None, patience=100, batch=4, imgsz=(1280, 720), save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train25, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None,

Fontconfig warning: ignoring UTF-8: not a valid region tag
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


Model summary: 295 layers, 25858636 parameters, 25858620 gradients, 79.1 GFLOPs

Transferred 469/475 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
WARNING ⚠️ updating to 'imgsz=1280'. 'train' and 'val' imgsz must be an integer, while 'predict' and 'export' imgsz may be a [h, w] list or an integer, i.e. 'yolo export imgsz=640,480' or 'yolo export imgsz=640'


train: Scanning /Users/kevin/Desktop/cv/final_project/obj_detection/datasets/labels/train.cache... 3407 images, 238 backgrounds, 0 corrupt: 100%|██████████| 3407/3407 [00:00<?, ?it/s]
val: Scanning /Users/kevin/Desktop/cv/final_project/obj_detection/datasets/labels/validation.cache... 425 images, 42 backgrounds, 0 corrupt: 100%|██████████| 425/425 [00:00<?, ?it/s]


Plotting labels to /Users/kevin/runs/detect/train25/labels.jpg... 
optimizer: Adam(lr=0.001, momentum=0.937) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 0 dataloader workers
Logging results to /Users/kevin/runs/detect/train25
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      1.627      2.309      1.114         28       1280:  45%|████▌     | 386/852 [18:18:37<21:17:39, 164.50s/it]